Object Detection Using YOLO

Set Up YOLOv5 Environment

In [1]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

c:\Users\USER\Documents\OPLearning\10_Academy\Week_7\notebooks\yolov5


Cloning into 'yolov5'...
Updating files:  47% (70/147)
Updating files:  48% (71/147)
Updating files:  49% (73/147)
Updating files:  50% (74/147)
Updating files:  51% (75/147)
Updating files:  52% (77/147)
Updating files:  53% (78/147)
Updating files:  54% (80/147)
Updating files:  55% (81/147)
Updating files:  56% (83/147)
Updating files:  57% (84/147)
Updating files:  58% (86/147)
Updating files:  59% (87/147)
Updating files:  60% (89/147)
Updating files:  61% (90/147)
Updating files:  62% (92/147)
Updating files:  63% (93/147)
Updating files:  64% (95/147)
Updating files:  65% (96/147)
Updating files:  66% (98/147)
Updating files:  67% (99/147)
Updating files:  68% (100/147)
Updating files:  69% (102/147)
Updating files:  70% (103/147)
Updating files:  71% (105/147)
Updating files:  72% (106/147)
Updating files:  73% (108/147)
Updating files:  74% (109/147)
Updating files:  75% (111/147)
Updating files:  76% (112/147)
Updating files:  77% (114/147)
Updating files:  78% (115/147)
Upda

In [2]:
import cv2
import os

image_folder = r"C:\Users\USER\Documents\OPLearning\10_Academy\Week_7\data\downloaded_images"


 Perform Object Detection and Save Results

In [3]:
import torch
import pandas as pd
import logging
import glob
import matplotlib.pyplot as plt
import hashlib
import shutil

os.makedirs('detection_results', exist_ok=True)
os.makedirs('selected_examples', exist_ok=True)

In [4]:

logging.basicConfig(filename='detection.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def compute_image_hash(image_path):
    with open(image_path, "rb") as image_file:
        buffer = image_file.read()
    hash_object = hashlib.md5(buffer)
    return hash_object.hexdigest()

In [5]:
unique_hashes = set()
duplicate_images = []
for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    image_hash = compute_image_hash(image_path)
    if image_hash in unique_hashes:
        duplicate_images.append(filename)
    else:
        unique_hashes.add(image_hash)

if duplicate_images:
    logging.warning(f"Found {len(duplicate_images)} duplicate images:")
    logging.warning(", ".join(duplicate_images))
else:
    logging.info("No duplicate images found.")

In [6]:
filtered_images = [img for img in os.listdir(image_folder) if img not in duplicate_images]
image_folder = image_folder
selected_images = filtered_images[:100]  

In [11]:
import torch

model = torch.hub.load("ultralytics/yolov5", "yolov5s", force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\USER/.cache\torch\hub\master.zip
YOLOv5  2024-10-14 Python-3.11.2 torch-2.4.1+cpu CPU

100%|██████████| 14.1M/14.1M [00:08<00:00, 1.75MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [12]:
for img_name in selected_images:
    try:
        img_path = os.path.join(image_folder, img_name)
        img = cv2.imread(img_path)

        if img is None:
            logging.warning(f"Warning: {img_name} could not be loaded.")
            continue

        results = model(img)

        shutil.copy2(img_path, 'selected_examples/')
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f"{img_name}")
        plt.savefig(f'selected_examples/{img_name}.png', bbox_inches='tight')
        plt.close()

        results.save(save_dir='detection_results/')

        detections = results.pandas().xyxy[0]
        csv_path = f'detection_results/{img_name}_detections.csv'
        detections[['name', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax']].to_csv(csv_path, index=False)

        logging.info(f"Successfully processed {img_name}")
    except Exception as e:
        logging.error(f"Error processing {img_name}: {str(e)}")

print("Detection completed.")


C:\Users\USER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 1 image to detection_results2
C:\Users\USER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 1 image to detection_results3
C:\Users\USER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 1 image to detection_results4
C:\Users\USER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` 

Detection completed.
